In [ ]:
import numpy as np

def simulate_oscillator():
    # Fixed, deterministic parameters
    m = 1.0
    c = 0.2
    k = 4.0
    x0 = 1.0
    v0 = 0.0
    dt = 0.001
    T = 10.0

    # Time grid
    N = int(np.round(T / dt))
    t = np.linspace(0.0, T, N + 1)

    # State arrays
    x = np.zeros_like(t)
    v = np.zeros_like(t)

    x[0] = x0
    v[0] = v0

    # ODE: m x'' + c x' + k x = 0  ->  x' = v,  v' = -(c/m) v - (k/m) x
    def f_state(x_, v_):
        dx = v_
        dv = -(c / m) * v_ - (k / m) * x_
        return dx, dv

    # RK4 time stepping
    for i in range(N):
        dx1, dv1 = f_state(x[i], v[i])

        dx2, dv2 = f_state(x[i] + 0.5 * dt * dx1, v[i] + 0.5 * dt * dv1)
        dx3, dv3 = f_state(x[i] + 0.5 * dt * dx2, v[i] + 0.5 * dt * dv2)
        dx4, dv4 = f_state(x[i] + dt * dx3, v[i] + dt * dv3)

        x[i+1] = x[i] + (dt / 6.0) * (dx1 + 2*dx2 + 2*dx3 + dx4)
        v[i+1] = v[i] + (dt / 6.0) * (dv1 + 2*dv2 + 2*dv3 + dv4)

    # Analytic underdamped solution for comparison
    # omega_n = sqrt(k/m), zeta = c/(2*sqrt(k m)), omega_d = omega_n*sqrt(1 - zeta^2)
    omega_n = np.sqrt(k / m)
    zeta = c / (2.0 * np.sqrt(k * m))
    assert zeta < 1.0  # underdamped
    omega_d = omega_n * np.sqrt(1.0 - zeta**2)

    # Analytic solution for x(t) given x(0)=x0, v(0)=v0
    # x(t) = e^{-zeta*omega_n*t} [ x0*cos(omega_d t) + ((v0 + zeta*omega_n*x0)/omega_d) * sin(omega_d t) ]
    A = x0
    B = (v0 + zeta * omega_n * x0) / omega_d
    x_analytic = np.exp(-zeta * omega_n * t) * (A * np.cos(omega_d * t) + B * np.sin(omega_d * t))

    # Error metric
    max_abs_error = float(np.max(np.abs(x - x_analytic)))

    # Energy: E = (1/2) m v^2 + (1/2) k x^2
    E = 0.5 * m * v**2 + 0.5 * k * x**2
    energy_ratio_end = float(E[-1] / E[0])

    # FFT-based dominant frequency estimate (Hz)
    # Use real FFT, ignore DC bin
    dt_s = dt
    fs = 1.0 / dt_s
    X = np.fft.rfft(x - np.mean(x))
    freqs = np.fft.rfftfreq(len(x), d=dt_s)
    # Exclude DC
    if len(freqs) > 1:
        idx = np.argmax(np.abs(X[1:])**2) + 1
    else:
        idx = 0
    peak_freq = float(freqs[idx])

    return {
        "t": t,
        "x": x,
        "v": v,
        "max_abs_error": max_abs_error,
        "energy_ratio_end": energy_ratio_end,
        "peak_freq": peak_freq,
    }

# Run once and show a brief summary
result = simulate_oscillator()
print({
    "len_t": len(result["t"]),
    "len_x": len(result["x"]),
    "len_v": len(result["v"]),
    "max_abs_error": result["max_abs_error"],
    "energy_ratio_end": result["energy_ratio_end"],
    "peak_freq": result["peak_freq"],
})